# Cassava Leaf Disease Modelling
There is already a great kernel [Vision Transformer (ViT): Tutorial + Baseline](https://www.kaggle.com/abhinand05/vision-transformer-vit-tutorial-baseline) that shows us how to use visiontransformer on TPU and why. Here, we make further improvements on the basis of the official implementation, in order to provide better pre-training parameters and user-friendly API as `Effecientnet-PyTorch`. You can find all the details on https://github.com/tczhangzhi/VisionTransformer-Pytorch.


In [ ]:
# !pip install vision_transformer_pytorch

In [ ]:
import sys

package_path = '../input/vision-transformer-pytorch/VisionTransformer-Pytorch'
sys.path.append(package_path)

In [ ]:
# Import libraries
import os
import pandas as pd
import albumentations as albu
import matplotlib.pyplot as plt
import json
import seaborn as sns
import cv2
import albumentations as albu
import numpy as np

In [ ]:
BASE_DIR="../input/cassava-leaf-disease-classification/"
TRAIN_IMAGES_DIR=os.path.join(BASE_DIR,'train_images')

train_df=pd.read_csv(os.path.join(BASE_DIR,'train.csv'))

In [ ]:
train_df.head()

In [ ]:
print("Count of training images {0}".format(len(os.listdir(TRAIN_IMAGES_DIR))))

In [ ]:
with open(f'{BASE_DIR}/label_num_to_disease_map.json', 'r') as f:
    name_mapping = json.load(f)
    
name_mapping = {int(k): v for k, v in name_mapping.items()}
train_df["class_id"]=train_df["label"].map(name_mapping)

In [ ]:
name_mapping

In [ ]:
len(train_df)

# Visualization

In [ ]:
def visualize_images(image_ids,labels):
    plt.figure(figsize=(16,12))
    
    for ind,(image_id,label) in enumerate(zip(image_ids,labels)):
        plt.subplot(3,3,ind+1)
        
        image=cv2.imread(os.path.join(TRAIN_IMAGES_DIR,image_id))
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        
        plt.imshow(image)
        plt.title(f"Class: {label}",fontsize=12)
        
        plt.axis("off")
    plt.show()
    

def plot_augmentation(image_id,transform):
    plt.figure(figsize=(16,4))
    
    img=cv2.imread(os.path.join(TRAIN_IMAGES_DIR,image_id))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    plt.subplot(1,3,1)
    plt.imshow(img)
    plt.axis("off")
    
    plt.subplot(1,3,2)
    x=transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")
    
    plt.subplot(1,3,3)
    x=transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")
    
    plt.show()
    
    
def visualize(images, transform):
    """
    Plot images and their transformations
    """
    fig = plt.figure(figsize=(32, 16))
    
    for i, im in enumerate(images):
        ax = fig.add_subplot(2, 5, i + 1, xticks=[], yticks=[])
        plt.imshow(im)
        
    for i, im in enumerate(images):
        ax = fig.add_subplot(2, 5, i + 6, xticks=[], yticks=[])
        plt.imshow(transform(image=im)['image'])

# Modelling

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, train_test_split
from albumentations.pytorch import ToTensorV2
# from efficientnet_pytorch import EfficientNet
import time
import datetime
import copy

In [ ]:
# DataSet class

class CassavaDataset(Dataset):
    def __init__(self,df:pd.DataFrame,imfolder:str,train:bool = True, transforms=None):
        self.df=df
        self.imfolder=imfolder
        self.train=train
        self.transforms=transforms
        
    def __getitem__(self,index):
        im_path=os.path.join(self.imfolder,self.df.iloc[index]['image_id'])
        x=cv2.imread(im_path,cv2.IMREAD_COLOR)
        x=cv2.cvtColor(x,cv2.COLOR_BGR2RGB)
        
        if(self.transforms):
            x=self.transforms(image=x)['image']
        
        if(self.train):
            y=self.df.iloc[index]['label']
            return x,y
        else:
            return x
        
    def __len__(self):
        return len(self.df)

In [ ]:
train_augs = albu.Compose([
    albu.RandomResizedCrop(height=384, width=384, p=1.0),
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),
    albu.RandomBrightnessContrast(p=0.5),
    albu.ShiftScaleRotate(p=0.5),
    albu.Normalize(    
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],),
    ToTensorV2(),
])

valid_augs = albu.Compose([
    albu.Resize(height=384, width=384, p=1.0),
    albu.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],),
    ToTensorV2(),
])

In [ ]:
train, valid = train_test_split(
    train_df, 
    test_size=0.1, 
    random_state=42,
    stratify=train_df.label.values
)


# reset index on both dataframes
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

train_targets = train.label.values

# targets for validation
valid_targets = valid.label.values

In [ ]:
train_dataset=CassavaDataset(
    df=train,
    imfolder=TRAIN_IMAGES_DIR,
    train=True,
    transforms=train_augs
)

valid_dataset=CassavaDataset(
    df=valid,
    imfolder=TRAIN_IMAGES_DIR,
    train=True,
    transforms=valid_augs
)

In [ ]:
def plot_image(img_dict):
    image_tensor = img_dict[0]
#     print(type(image_tensor))
    target = img_dict[1]
    print(target)
    plt.figure(figsize=(10, 10))
    image = image_tensor.permute(1, 2, 0) 
    plt.imshow(image)

In [ ]:
plot_image(train_dataset[5])

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    num_workers=4,
    shuffle=True,
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=16,
    num_workers=4,
    shuffle=False,
)


In [ ]:
def train_model(datasets, dataloaders, model, criterion, optimizer, scheduler, num_epochs, device):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels=labels.to(device)

                # Zero out the grads
                optimizer.zero_grad()

                # Forward
                # Track history in train mode
                with torch.set_grad_enabled(phase == 'train'):
                    model=model.to(device)
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # Statistics
                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss/len(datasets[phase])
            epoch_acc = running_corrects.double()/len(datasets[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time()-since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
from vision_transformer_pytorch import VisionTransformer

# model_name = 'efficientnet-b7'
datasets={'train':train_dataset,'valid':valid_dataset}
dataloaders={'train':train_loader,'valid':valid_loader}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model=models.(pretrained=True)
# model.fc=nn.Linear(512,5)
# model = EfficientNet.from_pretrained(model_name, num_classes=5) 
# model=models.resnext50_32x4d()#Add Pretrained=True to use pretrained with internet enabled
# model.fc=nn.Linear(model.fc.in_features,5)
model = VisionTransformer.from_name('ViT-B_16', num_classes=5) 

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
criterion=nn.CrossEntropyLoss()
num_epochs=6

Train the model after uncommenting below

In [ ]:
# trained_model=train_model(datasets,dataloaders,model,criterion,optimizer,scheduler,num_epochs,device)

In [ ]:
# Epoch 0/5
# ----------
# train Loss: 0.5318 Acc: 0.8119
# valid Loss: 0.4009 Acc: 0.8650

# Epoch 1/5
# ----------
# train Loss: 0.4384 Acc: 0.8467
# valid Loss: 0.3999 Acc: 0.8612

# Epoch 2/5
# ----------
# train Loss: 0.3511 Acc: 0.8778
# valid Loss: 0.3558 Acc: 0.8771

# Epoch 3/5
# ----------
# train Loss: 0.3266 Acc: 0.8879
# valid Loss: 0.3468 Acc: 0.8836

# Epoch 4/5
# ----------
# train Loss: 0.3066 Acc: 0.8924
# valid Loss: 0.3384 Acc: 0.8911

# Epoch 5/5
# ----------
# train Loss: 0.3060 Acc: 0.8926
# valid Loss: 0.3421 Acc: 0.8869

# Training complete in 121m 52s
# Best val Acc: 0.891121

Save the model after training

In [ ]:
# torch.save(model.state_dict(), 'ViT-B_16.pt')

Load the model when model is trained and saved and notebook has to be run without internet

In [ ]:
model.load_state_dict(torch.load('../input/vit-model-1/ViT-B_16.pt'))

# Submission

In [ ]:
test_df = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
image_path = "../input/cassava-leaf-disease-classification/test_images/"
# fake targets
test_targets = test_df.label.values


test_aug = albu.Compose([
            albu.CenterCrop(512, 512, p=1.),
            albu.Resize(384, 384),
            albu.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0),
            ToTensorV2()], p=1.)

test_dataset=CassavaDataset(
    df=test_df,
    imfolder=image_path,
    train=False,
    transforms=test_aug
)

test_loader =  DataLoader(
        test_dataset,
        batch_size=4,
        num_workers=4,
        shuffle=False,
)

In [ ]:
predictions=[]

for imgs in test_loader:

    imgs = imgs.to(device)
    with torch.no_grad():
        model=model.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        predicted=predicted.to('cpu')
        predictions.append(predicted)


In [ ]:
test_df['label'] = np.concatenate(predictions)


In [ ]:
test_df.to_csv('submission.csv', index=False)